In [0]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import tensorflow as tf



In [0]:
# data is not included due to NDA reasons. If there's request or need, please contact Dr. Peter Huang at (703) 589-4919
f = open("data_truck.txt","r")

t_dataX = []
t_dataY = []
for line in f:
  line = line.split(',')
  line.pop()
  line = [float(i) for i in line[5:]]
  
  #some filters for data to delete the anomalies
  if line[0] in (2,3,5):
    if line[1] > 1:
      if 200 < line[2] < 384:
        line = [int(i) for i in line]
        t_dataY.append(line[0])
        t_dataX.append([line[1]]+line[3:])

X = pd.DataFrame(t_dataX)
X = X.fillna(4)
Y = np.array(t_dataY)

In [0]:
#reserve test data using stratified fold, but only 1 fold is actually used.

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cv.get_n_splits(X,Y)
for train_index, test_index in cv.split(X, Y):
  x_train, x_test = X.values[train_index], X.values[test_index]
  y_train, y_test = Y[train_index], Y[test_index]


In [30]:
#random forest classifier
clf = RandomForestClassifier(n_estimators=200, max_depth=6, random_state=0)

#cross validation
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scoring = ['accuracy']
scores = cross_validate(clf, x_train, y_train, scoring=scoring,cv=cv)
print('Mean accuracy for random forest: ', scores['test_accuracy'].mean())

Mean accuracy for random forest:  0.9076547231270358


In [31]:
#adam solver is used in field data instead of lbfgs, since there is a large amount of data
clf2 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(30,), random_state=1)

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scoring = ['accuracy']
scores = cross_validate(clf2, x_train, y_train, scoring=scoring,cv=cv)
print('Mean accuracy for MLP classifier: ', scores['test_accuracy'].mean())

Mean accuracy for MLP classifier:  0.9120521172638437


In [26]:
#dense neural network by tensorflow training

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(384,)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.05),
  tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20)
model.evaluate(x_test, y_test)

W0730 16:08:32.256854 140352147982208 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/20
6137/6137 [==============================] - 2s 328us/sample - loss: 12.8822 - acc: 0.8305
Epoch 2/20
6137/6137 [==============================] - 2s 256us/sample - loss: 0.6746 - acc: 0.8841
Epoch 3/20
6137/6137 [==============================] - 2s 264us/sample - loss: 0.3143 - acc: 0.8941
Epoch 4/20
6137/6137 [==============================] - 2s 254us/sample - loss: 0.2719 - acc: 0.9092
Epoch 5/20
6137/6137 [==============================] - 2s 257us/sample - loss: 0.2395 - acc: 0.9136
Epoch 6/20
6137/6137 [==============================] - 2s 261us/sample - loss: 0.2462 - acc: 0.9112
Epoch 7/20
6137/6137 [==============================] - 2s 260us/sample - loss: 0.2384 - acc: 0.9159
Epoch 8/20
6137/6137 [==============================] - 2s 261us/sample - loss: 0.2283 - acc: 0.9158
Epoch 9/20
6137/6137 [==============================] - 2s 264us/sample - loss: 0.2267 - acc: 0.9138
Epoch 10/20
6137/6137 [==============================] - 2s 265us/sample - loss: 0.2285 - 

[0.17212067023162345, 0.93811077]